# Load_GeoClaw_fgout.ipynb

## Load fgout data from a netCDF file

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

Illustrating how to load a netCDF file containing GeoClaw fgout data, which consists of snapshots on a fixed output grid at a set of times.

For the example here, the fgout grid covers Seaside, OR at a spatial resolution of 1/3 arcsecond (about 7 meters in longitude, 10 m in latitude) and with a temporal resolution of 15 seconds, starting 20 minutes after the earthquake specified by `event` below.

This notebook illustrates how to read in the data an plot snapshots of the onshore inundation (as colormaps or along a transect).  It also shows how to extract a time series at a specific point, and compares these time series to those capture at gauges that were specified in advance of the GeoClaw run (which has better temporal resolution and is more accurate for a specific location, but extracting a time series from the fgout data is useful for looking at locations that were not specifed as gauges in advance).

This is a draft provided to solicit input on the format of these files.

A sample data file can be downloaded from 
[this SharePoint folder](https://uwnetid.sharepoint.com/sites/ptha/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2Fptha%2FShared%20Documents%2FTsunamiTestData&p=true&ga=1).

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import fgout_tools
import xarray
from clawpack.visclaw import geoplot, colormaps
from scipy.interpolate import RegularGridInterpolator

In [ ]:
event = 'buried-random-mur13-deep'
fgno = 3  # which fgout grid

In [ ]:
fname_nc = '%s-fgout%s.nc' % (event,str(fgno).zfill(3))
print('Will load fgout data from %s' % fname_nc)
print('You may need to download this file first from the folder specified at the top of this notebook')

In [ ]:
ncdata = xarray.open_dataset(fname_nc, decode_timedelta=False)

### Examine the data:

In [ ]:
ncdata

In [ ]:
ncdata.h

In [ ]:
ncdata.h.units

### Convert variables to numpy.ndarray's:

In [ ]:
t = asarray(ncdata.time)
x = asarray(ncdata.lon)
y = asarray(ncdata.lat)
            
# transpose multidimensional arrays if needed so [i,j,k] index corresponds to [x,y,t]:
B = asarray(ncdata.B0.transpose('lon','lat'))
#h = asarray(ncdata.h.transpose('lon','lat','time')) # call it depth instead
eta = asarray(ncdata.eta.transpose('lon','lat','time'))
depth = asarray(ncdata.h.transpose('lon','lat','time'))
u = asarray(ncdata.u.transpose('lon','lat','time'))
v = asarray(ncdata.v.transpose('lon','lat','time'))
speed = sqrt(u**2 + v**2)

X,Y = meshgrid(x,y,indexing='ij')
tmin = t / 60. # convert to minutes

ncdata.close()

In [ ]:
eta.shape, B.shape

### Create interpolation functions so we can evaluate at any (x,y,t):

In [ ]:
method = 'linear'
fill_value = nan
bounds_error = False # set value to nan if outside domain

eta_fcn = RegularGridInterpolator((x,y,t), eta,
                                  method=method, fill_value=fill_value, bounds_error=bounds_error)
depth_fcn = RegularGridInterpolator((x,y,t), depth,
                                    method=method, fill_value=fill_value, bounds_error=bounds_error)
speed_fcn = RegularGridInterpolator((x,y,t), speed,
                                    method=method, fill_value=fill_value, bounds_error=bounds_error)
B_fcn = RegularGridInterpolator((x,y), B,
                                method=method, fill_value=fill_value, bounds_error=bounds_error)

### Plot inundation at a couple times:

In [ ]:
land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})
sea_cmap = plt.get_cmap('Blues_r')

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=[-1,1],
                                     data_break=0)

In [ ]:
def plot_topo():
    fig,ax = subplots(figsize=(6,8))
    pcolormesh(X,Y,B,cmap=cmap)
    clim(-30,30)
    colorbar(extend='both',shrink=0.8,label='meters')
    ax.set_aspect(1/cos(pi*y.mean()/180))
    ticklabel_format(useOffset=False)
    xticks(rotation=20)
    title('Bathymetry / topography relative to MHW')

def plot_topo_contours(clines=[0.]):
    fig,ax = subplots(figsize=(6,8))
    contour(X,Y,B,clines,colors='g',linewidths=0.8)
    ax.set_aspect(1/cos(pi*y.mean()/180))
    ticklabel_format(useOffset=False)
    xticks(rotation=20)
    #title('Bathymetry / topography relative to MHW')

In [ ]:
def extract_grid_onshore(tt, make_plots=True):
    """
    extract depth and speed at time tt (minutes) at onshore points (nan offshore)
    if make_plots:  also plot depth on contour plot of topo
    """
    depth_t = depth_fcn((X,Y,tt*60))
    depth_onshore = where(B>0, depth_t, nan)
    speed_t = speed_fcn((X,Y,tt*60))
    speed_onshore = where(B>0, speed_t, nan)
    if make_plots:
        
        #cmap_reds = plt.get_cmap('Reds')
        cmap = plt.get_cmap('YlOrRd')
        #cmap.set_under([.9,1,.9])
        cmap.set_under([.1,.7,.1])
        cmap.set_over('m')
        plot_topo_contours()
        pcolormesh(X,Y,depth_onshore,cmap=cmap)
        clim(0.05,6)
        colorbar(extend='max',label='meters',shrink=0.8)
        title('Onshore depth at t = %.2f minutes' % tt)

    return depth_onshore, speed_onshore

In [ ]:
depth_onshore, speed_onshore = extract_grid_onshore(35)

In [ ]:
depth_onshore, speed_onshore = extract_grid_onshore(45)

## Extract some transects:

In [ ]:
y0 = 46.01 # transect location
figure(figsize=(10,6))
c = ['k','c','b']
for kc,kt in enumerate([0,50,100]):
    eta_y0 = eta_fcn((x,y0,t[kt]))
    plot(x, eta_y0, color=c[kc], label='surface at t = %.2f minutes' % tmin[kt])
B_y0 = B_fcn((x,y0))
plot(x, B_y0, 'g', label='topo at t = %.2f minutes' % tmin[0])
xlim(x[0],x[-1])
ylim(-10,10)
ticklabel_format(useOffset=False)
grid(True)
legend(loc='lower right', framealpha=1)
title('Transect at y = %.5f' % y0);

## Extract a time series at a point:

We can use the interpolating functions to evaluate depth, eta, B, etc. at an arbitrary point in the region covered by the fgout data.

### At an offshore point:

In [ ]:
xg = -123.9500
yg = 46.0100
tg = t # times from fgout
depth_tg = depth_fcn((xg,yg,tg))
eta_tg = eta_fcn((xg,yg,tg))
Bg = B_fcn((xg,yg))
figure(figsize=(10,4))
plot(t, eta_tg, 'b', label='eta_fgout')
plot(t, depth_tg, 'k', label='depth_fgout')
grid(True)
legend(loc='upper left', framealpha=1)
title('%s -- extracted time series at \nxg = %.5f, yg = %.5f, B = %.2f' % (event,xg,yg,Bg));

### At an onshore point:

In [ ]:
xg = -123.9300
yg = 46.0000
tg = t # times from fgout
depth_tg = depth_fcn((xg,yg,tg))
eta_tg = eta_fcn((xg,yg,tg))
Bg = B_fcn((xg,yg))

figure(figsize=(10,4))
plot(t, eta_tg, 'b', label='eta_fgout')
plot(t, depth_tg, 'k', label='depth_fgout')
grid(True)
legend(loc='upper left', framealpha=1)
title('%s -- extracted time series at \nxg = %.5f, yg = %.5f, B = %.2f' % (event,xg,yg,Bg));

## Compare time series from fgout to values captured at preset gauges:

For comparison purposes, read in all the gauge data calculated during the GeoClaw points, and
then select a point where there was a gauge.  This uses the function that is also defined in the notebook `Load_GeoClaw_GaugeSeries.ipynb`.

In [ ]:
from CHTtools import read_allgauges_nc
ncfile_gauges = 'allgauges_Seaside_18buried.nc'
print('Will read gauge solutions from %s' % ncfile_gauges)
print('You may need to download this file first from the folder specified at the top of this notebook')

In [ ]:
gauge_x, gauge_y, gauge_t, gauge_vals = read_allgauges_nc(ncfile_gauges)

In [ ]:
gaugeno = 1032
xg = float(gauge_x.sel(gaugeno=gaugeno))
yg = float(gauge_y.sel(gaugeno=gaugeno))
print('Gauge %i is at xg = %.5f, yg = %.5f' % (gaugeno,xg,yg))

In [ ]:
h_gauge = gauge_vals.sel(gaugeno=gaugeno, qoi='h', event=event)
print('h_gauge has %i time series values at gauge times gauge_t' % len(h_gauge))
B_gauge = gauge_vals.sel(gaugeno=gaugeno, qoi='eta', event=event) - h_gauge
print('B_gauge ranges between %.2f and %.2f meters' % (B_gauge.min(), B_gauge.max()))
print('initial B at gauge is %.2f meters, final B is %.2f meters' \
        % (B_gauge[0], B_gauge[-1]))

### Extract time series from fgout data set at this location:

First see if there happens to be an fgout point at exactly this location:

In [ ]:
depth_fgout = squeeze(asarray(depth[x==xg, y==yg]))

if len(depth_fgout) == 0:
    print('No depth_fgout data available at xg = %.8f, yg = %.8f' % (xg,yg))
    print('Will interpolate...')
    depth_fgout = depth_fcn((xg, yg, t))
    print('depth_fgout has %i time series values at fgout times t' % len(depth_fgout))

In [ ]:
depth_fgout = depth_fcn((xg, yg, t))
B_fgout = B_fcn((xg,yg))

figure(figsize=(10,6))
plot(gauge_t/60, h_gauge, 'c', linewidth=3, label='h_gauge')
plot(t/60, depth_fgout, 'b', label='depth_fgout')
grid(True)
xlabel('time (minutes)')
ylabel('meters')
legend(loc='upper left', framealpha=1)
title('%s time series at Gauge %i\nxg = %.5f, yg = %.5f, Bg = %.2fm' \
          % (event,gaugeno,xg,yg,B_fgout));

fgout data was captured starting only at t = 20 minutes, whereas the gauge data was captured starting at time 0.

Note that the gauge data has better temperal resolution than the fgout data, and also the interpolation in space causes some differences.

We can see this better if we zoom in around time time of maximum depth:

In [ ]:
figure(figsize=(10,6))
plot(gauge_t/60, h_gauge, 'c-+', linewidth=3, label='h_gauge')
plot(t/60, depth_fgout, 'b-x', label='depth_fgout')
xlim(40,45)
ylim(12,20)
grid(True)
xlabel('time (minutes)')
ylabel('meters')
legend(loc='upper left', framealpha=1)
title('%s time series at Gauge %i\nxg = %.5f, yg = %.5f' % (event,gaugeno,xg,yg));

### Plots of surface eta:

Similar to above, but now plotting the surface elevation...

In [ ]:
eta_gauge =  gauge_vals.sel(gaugeno=gaugeno, qoi='eta', event=event)
eta_fgout = eta_fcn((xg, yg, t))

figure(figsize=(10,6))
plot(gauge_t/60, eta_gauge, 'c', linewidth=3, label='eta_gauge')
plot(t/60, eta_fgout, 'b', label='eta_fgout')
grid(True)
xlabel('time (minutes)')
ylabel('meters')
legend(loc='upper left', framealpha=1)
title('%s time series at Gauge %i\nxg = %.5f, yg = %.5f' % (event,gaugeno,xg,yg))

### For an onshore location:

In [ ]:
gaugeno = 1045

# from the gauges output:
xg = float(gauge_x.sel(gaugeno=gaugeno))
yg = float(gauge_y.sel(gaugeno=gaugeno))
print('Gauge %i is at xg = %.5f, yg = %.5f' % (gaugeno,xg,yg))
h_gauge = gauge_vals.sel(gaugeno=gaugeno, qoi='h', event=event)
print('h_gauge has %i time series values at gauge times gauge_t' % len(h_gauge))

# interpolating from fgout:
depth_fgout = depth_fcn((xg, yg, t))
print('depth_fgout has %i time series values at fgout times t' % len(depth_fgout))
B_fgout = B_fcn((xg,yg))

figure(figsize=(10,6))
plot(gauge_t/60, h_gauge, 'c', linewidth=3, label='h_gauge')
plot(t/60, depth_fgout, 'b', label='depth_fgout')
grid(True)
xlim(35,60)
xlabel('time (minutes)')
ylabel('meters')
legend(loc='upper left', framealpha=1)
title('%s time series at Gauge %i\nxg = %.5f, yg = %.5f, Bg = %.2fm' \
          % (event,gaugeno,xg,yg,B_fgout));